In [1]:
import os
os.chdir("..")

In [3]:
%pwd

'/Users/yashlucky/Code/MLProjectsYoutube/chickenDiseaseClassification-mlproject'

In [4]:
import tensorflow as tf

In [32]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: list
    training_dataframe_path: Path


In [23]:
from src.cdclassifier.constants import *
from src.cdclassifier.utils.common import read_yaml, create_directories, save_json

In [28]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            param_filepath = PARAMS_FILE_PATH
        ):
        
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_validation_config(self)-> EvaluationConfig:
        config = self.config.model_evaluation
        eval_config = EvaluationConfig(
            path_of_model=Path(config.trained_model_path),
            training_data=Path(config.training_data),
            all_params=self.param,
            params_image_size=self.param.IMAGE_SIZE,
            params_batch_size=self.param.BATCH_SIZE,
            training_dataframe_path=Path(config.training_dataframe_path)
        )
        return eval_config

In [29]:
from urllib.parse import urlparse
import pandas as pd

In [30]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):
        df = pd.read_csv(self.config.training_dataframe_path)

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_dataframe(
            dataframe=df,
            directory=self.config.training_data,
            x_col='images',
            y_col="label",
            subset="validation",
            seed=42,
            shuffle=True,
            class_mode="categorical",
            classes=['Salmonella', 'Coccidiosis', 'New Castle Disease', 'Healthy'],
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [31]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(config=val_config)
    evaluation.evaluation()
    evaluation.save_score()
     
except Exception as e:
    raise e

[2024-08-19 03:40:09,901: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-19 03:40:09,903: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-19 03:40:09,904: INFO: common: created directory at: artifacts]
Found 2420 validated image filenames belonging to 4 classes.
152/152 [==============================] - 129s 847ms/step - loss: 1.6411 - accuracy: 0.8120
[2024-08-19 03:42:18,973: INFO: common: json file saved at: scores.json]
